In [5]:
import os

In [6]:
%pwd

'd:\\Profiles\\sg2755\\Desktop\\New folder\\chicken_disease_classification_project\\research'

In [7]:
os.chdir("../")

In [8]:
%pwd

'd:\\Profiles\\sg2755\\Desktop\\New folder\\chicken_disease_classification_project'

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir:Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path
    

In [14]:
from src.CNN_Classifier.constant import *
from src.CNN_Classifier.utils.common import read_yaml,create_directory

In [39]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directory([self.config.artifacts_root])
        
    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directory([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])
        
        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        
        return prepare_callback_config

In [40]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [41]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config
    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only = True
        )
    
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]
        

In [42]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
except Exception as e:
    raise e

[2024-08-24 13:59:11,179: INFO: common: yaml file: config\config.yaml]
[2024-08-24 13:59:11,183: INFO: common: yaml file: params.yaml]
[2024-08-24 13:59:11,185: INFO: common: Created directory at:artifacts]
[2024-08-24 13:59:11,189: INFO: common: Created directory at:artifacts\prepare_callbacks\checkpoint_dir]
[2024-08-24 13:59:11,192: INFO: common: Created directory at:artifacts\prepare_callbacks\tensorboard_log_dir]
